In [ ]:
import pandas as pd
import geopandas as gpd
from sqlalchemy import create_engine, text
import sys

# --- 1. CONFIGURAÇÕES ---
db_params = {
    'user': 'USER',
    'password': 'SENHA',
    'host': 'HOST',
    'port': 'PORT',
    'database': 'postgres'
}

caminho_shapefile_geometria = '../data/BR_bairros_CD2022/BR_bairros_CD2022.shp'

# --- Chave de junção conforme solicitado ---
chave_juncao = ['CD_BAIRRO', 'CD_MUN', 'NM_BAIRRO']

caminho_saida_shapefile = '../shapefile_output/tipologia_bairros/tipologia_bairros.shp'

# --- 2. CONECTAR AO POSTGRESQL E LER A VIEW ---
df_analise = None
print("Conectando ao banco de dados e lendo a view de análise...")

try:
    engine_str = (
        f"postgresql+psycopg2://{db_params['user']}:{db_params['password']}@"
        f"{db_params['host']}:{db_params['port']}/{db_params['database']}"
    )
    engine = create_engine(
        engine_str,
        connect_args={'client_encoding': 'latin1'}
    ) 
    
    with engine.connect() as connection:
        # CORREÇÃO: Envolve o comando SQL na função text()
        connection.execute(text("SET client_encoding TO 'latin1';"))
        
        sql_query = 'SELECT * FROM cruzamentos.analise_tipologia_bairros'
        df_analise = pd.read_sql(sql_query, connection)
        
    print(f"{len(df_analise)} registros lidos com sucesso da view 'analise_tipologia_bairros'.")

except Exception as e:
    print(f"ERRO CRÍTICO ao conectar ou ler do banco de dados: {e}")
    sys.exit()

# --- 3. LER E PREPARAR O SHAPEFILE ---
print(f"Lendo o shapefile de geometria: {caminho_shapefile_geometria}")
try:
    gdf_geometria = gpd.read_file(caminho_shapefile_geometria)
    print("Colunas encontradas no shapefile:", gdf_geometria.columns.tolist())
    print(f"{len(gdf_geometria)} registros lidos do shapefile.")
except Exception as e:
    print(f"ERRO ao ler ou processar o shapefile: {e}")
    sys.exit()


# --- 4. JUNTAR OS DADOS (MERGE) ---
print(f"Preparando para juntar os dados usando as chaves: {chave_juncao}...")

try:
    for col in chave_juncao:
        print(f"Convertendo a coluna '{col}' para string em ambos os dataframes.")
        gdf_geometria[col] = gdf_geometria[col].astype(str)
        df_analise[col] = df_analise[col].astype(str)
except KeyError as e:
    print(f"ERRO DE CHAVE: A coluna {e} não foi encontrada em um dos dataframes. Verifique se os nomes correspondem à 'chave_juncao'.")
    sys.exit()

# A junção agora é entre o GeoDataFrame e o DataFrame normal
gdf_final = gdf_geometria.merge(
    df_analise,
    on=chave_juncao,
    how='inner' 
)

sucesso_join = len(gdf_final)
print(f"{sucesso_join} bairros tiveram correspondência e foram incluídos no resultado final.")
if sucesso_join == 0:
    print("AVISO: Nenhum bairro teve correspondência. Verifique os códigos de bairro e município.")
    sys.exit()


# --- 5. FILTRAR E SALVAR O SHAPEFILE FINAL ---
print("Preparando as colunas finais para o shapefile...")
try:
    # CORREÇÃO: Adicionadas as colunas de contagem e divisão público/privado
    colunas_para_manter = [
        'geometry', 'CD_BAIRRO', 'NM_BAIRRO', 'CD_MUN', 'NM_MUN',
        'renda_per_capita', 'populacao_total', 'categoria_bairro',
        'taxa_cobertura_infantil', 'taxa_cobertura_fundamental', 'taxa_cobertura_medio',
        'num_escolas', 'num_escolas_priv', 'mat_total_priv', 'mat_total_pub'
    ]
    gdf_filtrado = gdf_final[colunas_para_manter]

    # CORREÇÃO: Renomeia as novas colunas e as anteriores para nomes com 10 caracteres ou menos
    colunas_renomeadas = {
        'renda_per_capita': 'renda_pc',
        'populacao_total': 'pop_total',
        'categoria_bairro': 'categoria',
        'taxa_cobertura_infantil': 'tx_cob_inf',
        'taxa_cobertura_fundamental': 'tx_cob_fun',
        'taxa_cobertura_medio': 'tx_cob_med',
        'num_escolas': 'num_esc',
        'num_escolas_priv': 'num_esc_pr',
        'mat_total_priv': 'mat_tot_pr',
        'mat_total_pub': 'mat_tot_pu'
    }
    gdf_final_renomeado = gdf_filtrado.rename(columns=colunas_renomeadas)

except KeyError as e:
    print(f"ERRO: A coluna {e} não foi encontrada no dataframe final. Verifique os nomes das colunas na sua VIEW.")
    sys.exit()


# --- 6. CRIAR DIRETÓRIO E SALVAR O SHAPEFILE ---
print(f"Salvando o shapefile final em: {caminho_saida_shapefile}")
try:
    gdf_final_renomeado.to_file(caminho_saida_shapefile, driver='ESRI Shapefile', encoding='utf-8')
    print("\nProcesso concluído com sucesso!")
    print("O shapefile com a tipologia dos bairros está pronto para ser aberto em um software GIS.")
except Exception as e:
    print(f"ERRO ao salvar o shapefile: {e}")
